In [1]:
!pip install h5py
!pip install pandas
!pip install tsfresh


In [16]:
import h5py
import pandas as pd
from tsfresh import extract_features, select_features
from tsfresh.feature_extraction import EfficientFCParameters
import multiprocessing
import os
import pickle
import re

In [ ]:

with h5py.File("test.h5", "r") as f:
    # Explorar las claves disponibles
    print("Keys (datasets disponibles):", list(f.keys()))
    # Cargar datos de 'x' y 'y'
    x = f['x'][:]
    
    print("Shape de 'x':", x.shape) # 1 representa el canal por el caul se tomo
    print("Tipos de datos de 'x': ", x.dtype)


Keys (datasets disponibles): ['x', 'y']
Shape de 'x': (204, 1, 18530)
Tipos de datos de 'x':  float64
Shape de 'y': (204,)
Tipos de datos de 'y':  int64


In [3]:
with h5py.File("test.h5", "r") as f_pd:
    x_data = f_pd['x'][:]  
x_df = pd.DataFrame(x_data[:,0,:])  

print("Shape de 'x':", x_df.shape) 

# Ver las primeras filas del DataFrame
x_df.head()


Shape de 'x': (205, 18530)


,0,1,2,3,4,5,6,7,8,9,...,18520,18521,18522,18523,18524,18525,18526,18527,18528,18529
0,0.029938,0.061432,0.072388,0.081329,0.078033,0.089691,0.071564,0.072205,0.062988,0.057098,...,-0.000580,0.001617,0.003540,0.003662,0.002411,0.003143,0.002594,0.002045,0.002594,0.000549
1,0.000275,0.001160,0.001556,0.001740,0.002380,0.003693,0.004883,0.005310,0.005676,0.005310,...,0.013000,0.009399,0.005981,0.001801,-0.002045,-0.006378,-0.010986,-0.015106,-0.019196,-0.022736
2,-0.002899,-0.003510,0.000458,0.002991,0.005920,0.007355,0.008728,0.008698,0.007507,0.005341,...,-0.014679,-0.019745,-0.022583,-0.023346,-0.020325,-0.014557,-0.006927,0.001343,0.008972,0.015472
3,0.008636,0.018158,0.015717,0.015991,0.015442,0.015564,0.014679,0.014923,0.013916,0.014008,...,0.007874,0.006989,0.006531,0.006012,0.005066,0.005646,0.004944,0.005127,0.003754,0.003174
4,-0.011475,-0.022217,-0.017456,-0.018890,-0.019714,-0.021301,-0.025055,-0.033569,-0.039215,-0.038483,...,0.007263,0.008270,0.010071,0.012146,0.014221,0.015442,0.015778,0.015320,0.015198,0.015656


In [4]:
# Parseamos el dataset para poder usar la libreía de las series temporales

def parsear_time_series(dataset):
    parsed = []  

    for id, series in enumerate(dataset.values):
        # Aplanar la serie (la serie tiene forma (1, N), la aplanamos)
        series = series.flatten()

        # Crear el DataFrame para cada serie temporal
        df = pd.DataFrame({
            'id': id,  # Identificador de la serie
            'time': range(len(series)),  # Tiempo 
            'valor': series  # Los valores de la serie temporal
        })
        parsed.append(df) 

    # Concatenamos todos los DataFrames en uno solo
    result = pd.concat(parsed, ignore_index=True)
    
    return result

    

In [5]:
df_ready = parsear_time_series(x_df)

print("EL shape de df_rady para la extracción es: ", df_ready.shape)
print(df_ready.dtypes)
print(df_ready.head())


EL shape de df_rady para la extracción es:  (3798650, 3)
id         int64
time       int64
valor    float64
dtype: object
   id  time     valor
0   0     0  0.029938
1   0     1  0.061432
2   0     2  0.072388
3   0     3  0.081329
4   0     4  0.078033


In [7]:
import multiprocessing
print("Número de CPUs disponibles:", multiprocessing.cpu_count())

Número de CPUs disponibles: 16


In [26]:


# Montamos Google Drive

# Definimos la ruta de salida
output_dir = "featuresTest"
print(os.listdir(output_dir))

# Iterar sobre cada ID único en el DataFrame
unique_ids = df_ready["id"].unique()

# Procesar cada ID individualmente
for id_value in unique_ids:
    print(f"Procesando ID: {id_value}...")

    # Filtrar el DataFrame para solo ese ID
    df_id = df_ready[df_ready["id"] == id_value]

    # Extraer características para ese ID
    features_id = extract_features(df_id,
                                   column_id="id",
                                   column_sort="time",
                                   default_fc_parameters=EfficientFCParameters(),
                                   disable_progressbar=False,
                                   n_jobs=multiprocessing.cpu_count())
    # Rellenamos con 0 los NaN
    features_id = features_id.fillna(0)
    print("EL tamaño de feature id es: ",features_id.shape)

    # Guardar las características de ese ID en un archivo
    features_id_path = os.path.join(output_dir, f"features_id_{id_value}.pkl")

    features_id.to_pickle(features_id_path)

    print(f"Características de ID={id_value} guardadas en: {features_id_path}")


[]
Procesando ID: 0...


Feature Extraction: 100%|██████████| 1/1 [00:03<00:00,  3.74s/it]

EL tamaño de feature id es:  (1, 777)
Características de ID=0 guardadas en: featuresTest/features_id_0.pkl
Procesando ID: 1...



Feature Extraction: 100%|██████████| 1/1 [00:03<00:00,  3.51s/it]

EL tamaño de feature id es:  (1, 777)
Características de ID=1 guardadas en: featuresTest/features_id_1.pkl
Procesando ID: 2...


...

EL tamaño de feature id es:  (1, 777)
Características de ID=202 guardadas en: featuresTest/features_id_202.pkl
Procesando ID: 203...



Feature Extraction: 100%|██████████| 1/1 [00:03<00:00,  3.94s/it]

EL tamaño de feature id es:  (1, 777)
Características de ID=203 guardadas en: featuresTest/features_id_203.pkl
Procesando ID: 204...



Feature Extraction: 100%|██████████| 1/1 [00:03<00:00,  3.48s/it]

EL tamaño de feature id es:  (1, 777)
Características de ID=204 guardadas en: featuresTest/features_id_204.pkl


In [27]:


# Ruta donde están guardados los archivos de características
input_dir = "featuresTest"
output_dir = "featuresTest/featuresTestAll"
#os.makedirs(output_dir, exist_ok=True)
# Montamos Google Drive
# Lista para almacenar cada DataFrame cargado
features_list = []

def extract_id_number(filename):
    """Se encarga de buscar el id del nombre del archivo para procesar de manera ordenada"""
    match = re.search(r"features_id_(\d+)\.pkl", filename)
    return int(match.group(1)) if match else -1

files = [f for f in os.listdir(input_dir) if f.startswith("features_id_") and f.endswith(".pkl")]
files_sorted = sorted(files, key=extract_id_number)

features_list = []
for file in files_sorted:
    print(f"Procesando archivo: {file}...")
    df = pd.read_pickle(os.path.join(input_dir, file))
    features_list.append(df)


# Concatenamos todos los DataFrames en uno solo
full_features_df = pd.concat(features_list, axis=0)

full_features_df.reset_index(drop=True, inplace=True)

full_features_df.to_pickle(os.path.join(output_dir, "all_features_test.pkl"))
# Mostrar información del DataFrame final
print("Shape del dataset final es: ", full_features_df.shape)
full_features_df.head()



Procesando archivo: features_id_0.pkl...
Procesando archivo: features_id_1.pkl...
Procesando archivo: features_id_2.pkl...
Procesando archivo: features_id_3.pkl...
Procesando archivo: features_id_4.pkl...
Procesando archivo: features_id_5.pkl...
Procesando archivo: features_id_6.pkl...
Procesando archivo: features_id_7.pkl...
Procesando archivo: features_id_8.pkl...
Procesando archivo: features_id_9.pkl...
Procesando archivo: features_id_10.pkl...
Procesando archivo: features_id_11.pkl...
Procesando archivo: features_id_12.pkl...
Procesando archivo: features_id_13.pkl...
Procesando archivo: features_id_14.pkl...
Procesando archivo: features_id_15.pkl...
Procesando archivo: features_id_16.pkl...
Procesando archivo: features_id_17.pkl...
Procesando archivo: features_id_18.pkl...
Procesando archivo: features_id_19.pkl...
Procesando archivo: features_id_20.pkl...
Procesando archivo: features_id_21.pkl...
Procesando archivo: features_id_22.pkl...
Procesando archivo: features_id_23.pkl...
Pr

,valor__variance_larger_than_standard_deviation,valor__has_duplicate_max,valor__has_duplicate_min,valor__has_duplicate,valor__sum_values,valor__abs_energy,valor__mean_abs_change,valor__mean_change,valor__mean_second_derivative_central,valor__median,...,valor__fourier_entropy__bins_5,valor__fourier_entropy__bins_10,valor__fourier_entropy__bins_100,valor__permutation_entropy__dimension_3__tau_1,valor__permutation_entropy__dimension_4__tau_1,valor__permutation_entropy__dimension_5__tau_1,valor__permutation_entropy__dimension_6__tau_1,valor__permutation_entropy__dimension_7__tau_1,valor__query_similarity_count__query_None__threshold_0.0,valor__mean_n_absolute_max__number_of_maxima_7
0,0.0,0.0,0.0,1.0,-1.108182,188.142145,0.008619,-1.586077e-06,-9.050847e-07,-0.000534,...,0.249958,0.350689,0.709399,1.088459,1.524412,1.997020,2.493913,3.009824,0.0,0.544525
1,0.0,0.0,0.0,1.0,-2.736053,2.089645,0.000988,-1.241851e-06,-1.194152e-07,-0.000214,...,0.318449,0.418924,0.741638,1.330092,1.950442,2.631909,3.359304,4.056247,0.0,0.076551
2,0.0,0.0,0.0,1.0,-0.200805,8.075229,0.002489,9.915041e-07,1.918878e-07,-0.000031,...,0.344487,0.482843,1.042906,1.168164,1.679338,2.239246,2.828654,3.425464,0.0,0.126382
3,0.0,0.0,0.0,1.0,-0.108185,7.379380,0.001721,-2.948161e-07,-2.725960e-07,0.000305,...,0.341853,0.404122,0.730892,1.336187,1.940234,2.583686,3.257647,3.895326,0.0,0.125427
4,0.0,0.0,0.0,1.0,-0.873899,32.803556,0.003637,1.464198e-06,3.022442e-07,0.000244,...,0.249958,0.294982,0.730892,1.195760,1.747804,2.355175,2.996904,3.639787,0.0,0.284114
